# Question 3

In [147]:
import numpy as np

In [148]:
B = 100 + 1/32
n = 10
c = 3.375
x0 = .1
tol = 1e-6

t = [x*.5 for x in range(1,n+1)]
v = [c/2 for x in range(n)]
v[n-1] += 100

In [149]:
def newton(x, t, v, B):
    num_sum = 0
    den_sum = 0
    
    for a, b in zip(t,v):
        # print(b, b*np.exp(-x*a))
        num_sum += b*np.exp(-x*a)
    num_sum -= B
        
    for c, d in zip(t,v):
        # print(d, c*d*np.exp(-x*c))
        den_sum += c*d*np.exp(-x*c)
    
    x_new = x + num_sum / den_sum
    
    
    return x_new

In [150]:
x_new = x0
x_old = x0-1
while np.abs(x_new-x_old) > tol:
    x_old = x_new
    x_new = newton(x_old, t, v, B)
    print(x_old, x_new)

0.1 0.021397014741595874
0.021397014741595874 0.033057328860338325
0.033057328860338325 0.03340081140223473
0.03340081140223473 0.03340109853803073


In [151]:
D = 0 
C = 0

for c, d in zip(t,v):
    # print(d, c*d*np.exp(-x*c))
    D += c*d*np.exp(-x_new*c)
    C += c**2*d*np.exp(-x_new*c)
    
D /= B
C /= B

print(f"Yield: {x_new*100:1f}%")
print(f"Duration: {D}")
print(f"Convexity: {C}")

Yield: 3.340110%
Duration: 4.642735048429879
Convexity: 22.573118015872677


# Question 5

In [152]:
from scipy.stats import norm

tol = 1e-7

def fxn(x):
    T = .25
    K = 30
    S = 30
    q = .02
    C = 2.5
    r = .06
    
    d1 = (np.log(S/K)+(r-q+.5*x**2)*T)/(x*np.sqrt(T))
    d2 = d1 - x*np.sqrt(T)
    
    C_approx = S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    delta = C-C_approx
    
    return delta

In [153]:
# Bisection method
a, b = .0001, 1
tol_int = tol
tol_approx = tol

def bisect(a, b, tol_int, tol_approx):
    x_l = a
    x_r = b
    i = 0 
    
    while max(abs(fxn(x_l)),abs(fxn(x_r))) > tol_approx or (x_r-x_l)>tol_int:
        x_m = (x_l + x_r)/2
        print(f"{i} || ", x_l, x_r)
        if fxn(x_l)*fxn(x_m) < 0:
            x_r = x_m
        else:
            x_l = x_m
        i += 1
    
    return x_m
    


In [154]:
bisect(a, b, tol_int, tol_approx)

0 ||  0.0001 1
1 ||  0.0001 0.50005
2 ||  0.250075 0.50005
3 ||  0.37506249999999997 0.50005
4 ||  0.37506249999999997 0.43755625
5 ||  0.37506249999999997 0.406309375
6 ||  0.39068593749999997 0.406309375
7 ||  0.39068593749999997 0.39849765625
8 ||  0.394591796875 0.39849765625
9 ||  0.3965447265625 0.39849765625
10 ||  0.3965447265625 0.39752119140625
11 ||  0.39703295898437496 0.39752119140625
12 ||  0.39703295898437496 0.39727707519531247
13 ||  0.39703295898437496 0.3971550170898437
14 ||  0.39703295898437496 0.3970939880371093
15 ||  0.39703295898437496 0.3970634735107421
16 ||  0.39704821624755854 0.3970634735107421
17 ||  0.39704821624755854 0.39705584487915035
18 ||  0.39704821624755854 0.3970520305633545
19 ||  0.39704821624755854 0.39705012340545653
20 ||  0.39704821624755854 0.3970491698265075
21 ||  0.39704821624755854 0.39704869303703305
22 ||  0.39704845464229577 0.39704869303703305
23 ||  0.3970485738396644 0.39704869303703305
24 ||  0.3970485738396644 0.39704863343834

0.3970486185386777

In [155]:
# Newton method
x0 = .5
tol_consec = tol

def d_fxn(x):
    T = .25
    K = 30
    S = 30
    q = .02
    C = 2.5
    r = .06
    
    d1 = (np.log(S/K)+(r-q+.5*x**2)*T)/(x*np.sqrt(T))
    
    y = -S*np.exp(-q*T)*np.sqrt(T/2/np.pi)*np.exp(-.5*d1**2)
    
    return y

def newton1(x0, tol_consec):
    x_new = x0
    x_old = 0
    i = 0
    
    while abs(fxn(x_new)) > tol_approx or abs(x_new-x_old) > tol_consec:

        x_old = x_new
        x_new = x_old - (fxn(x_old) / d_fxn(x_old))
        print(f"{i} || ", x_old, x_new,fxn(x_old),d_fxn(x_old))

        
        i+=1
        
    return x_new


In [156]:
newton1(x0, tol_consec)

0 ||  0.5 0.39691563052053713 -0.6054953855003937 -5.873784634449595
1 ||  0.39691563052053713 0.39704862856860385 0.0007830945397895306 -5.888015284228571
2 ||  0.39704862856860385 0.39704862873147384 9.58978674248101e-10 -5.888000861400251


0.39704862873147384

In [157]:
# Secant Method
x_1, x0 = .6, .5

def secant(x0, x_1):
    x_new = x0
    x_old = x_1
    i = 0
    
    while abs(fxn(x_new))>tol_approx or abs(x_new-x_old)>tol_consec:
        x_oldest = x_old
        x_old = x_new
        x_new = x_old - fxn(x_old)*(x_old-x_oldest)/(fxn(x_old)-fxn(x_oldest))
        print(f"{i} || ", x_old, x_new)
        
        i += 1
        
    return x_new


In [158]:
secant(x0, x_1)

0 ||  0.5 0.3967574905209847
1 ||  0.3967574905209847 0.3970489566893094
2 ||  0.3970489566893094 0.3970486287323524
3 ||  0.3970486287323524 0.3970486287314739


0.3970486287314739

In [159]:
fxn(0.3970486287314739)

-3.552713678800501e-15

In [160]:
# (ii) sig,approx
T = .25
K = 30
S = 30
q = .02
C = 2.5
r = .06
sig_approx = np.sqrt(2*np.pi/T)/S*(C-(r-q)*T*S/2)/(1-(r+q)*T/2)
sig_imp = newton1(x0, tol_consec)

0 ||  0.5 0.39691563052053713 -0.6054953855003937 -5.873784634449595
1 ||  0.39691563052053713 0.39704862856860385 0.0007830945397895306 -5.888015284228571
2 ||  0.39704862856860385 0.39704862873147384 9.58978674248101e-10 -5.888000861400251


In [161]:
rel_error = abs(sig_approx - sig_imp)/sig_imp
sig_approx, sig_imp, f"{rel_error*100:1f}% error"

(0.39667181450389566, 0.39704862873147384, '0.094904% error')

# Question 6

In [162]:
x01, x02 = np.array([1.0, 2.0, 3.0]), np.array([2.0, 2.0, 2.0])
tol_approx = 1e-9
tol_consec = 1e-6
h = 1e-6

def f (x):
    x1, x2, x3 = x
    
    y1 = x1**3 + 2*x1*x2 + x3**2 - x2*x3 + 9
    y2 = 2*x1**2 + 2*x1*x2**2 + (x2**3)*x3**2 - (x2**2)*x3 - 2
    y3 = x1*x2*x3 + x1**3 - x3**2 - x1*x2**2 - 4
    
    return np.array([y1, y2, y3])

f(x01), f(x02)

(array([ 17.,  68., -10.]), array([25., 46.,  0.]))

In [163]:
x01

array([1., 2., 3.])

In [164]:
def d_f(x,h):

    a = []

    for j in range(3):

        x_addh = np.copy(x)
        x_minh = np.copy(x)

        x_addh[j] = x[j] + h
        x_minh[j] = x[j] - h
        diff = (f(x_addh)-f(x_minh))/h/2
        a.append(diff)

    D_f = np.array(a).T

    return D_f

d_f(x01,h)

array([[  7.        ,  -1.        ,   4.        ],
       [ 12.        , 104.00000001,  44.        ],
       [  5.        ,  -1.        ,  -4.        ]])

In [165]:
x_new = x01
x_old = x01 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_f(x_old, h)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [1. 2. 3.]
2  || [ 0.5464  2.7784 -0.2616]
3  || [-2.45415693  7.13392935 -0.4031109 ]
4  || [-1.59789098  4.85550587 -0.37855094]
5  || [-1.42550577  3.00500187 -0.45121636]
6  || [-1.62369253  1.99376511 -0.67567152]
7  || [-1.69598454  1.97436714 -0.90336961]
8  || [-1.69054198  1.98302968 -0.88451335]
9  || [-1.69055076  1.98310724 -0.88455808]


array([-1.69055076,  1.98310724, -0.88455808])

In [166]:
x_new = x02
x_old = x02 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_f(x_old, h)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [2. 2. 2.]
2  || [-0.4516129  -5.35483869 16.4677419 ]
3  || [ 0.21571499 -5.66418295  6.75906942]
4  || [ 0.28602268 -6.52533236  1.79625249]
5  || [ -0.25842516 -12.48640205  -1.61432307]
6  || [ -0.08943886 -11.02414739  -1.1070591 ]
7  || [  0.72871716 -46.46471468   4.67295909]
8  || [ -1.10342382 -88.17587469  -3.96511324]
9  || [ -4.72168389  90.39758575 -14.02412521]
10  || [ 60.64566635 781.02235277 153.71619479]
11  || [ 42.27046645 536.00994183 149.19112111]
12  || [ 30.58973864 370.12045352 143.85547183]
13  || [ 22.5721881  260.76222826 135.68485421]
14  || [ 16.41145569 189.53256107 123.43840836]
15  || [ 11.38874428 141.72802823 108.42112642]
16  || [  7.1708098  107.94280234  92.98004836]
17  || [ 3.26698209 82.65719353 79.16268098]
18  || [-3.57933022 60.38585924 71.57901487]
19  || [-0.30292914 51.0528489  52.38729604]
20  || [ 7.10091581 46.62257107 33.01456459]
21  || [ 4.17451236 35.05882305 28.79374208]
22  || [ 1.69067053 26.05095166 25.49984672]
23  || [-9

array([-1.,  3.,  1.])

In [167]:
tol_approx, tol_consec

(1e-09, 1e-06)

In [168]:
# forward differences
def d_ff(x,h):

    a = []

    for j in range(3):

        x_addh = np.copy(x)
        x_minh = np.copy(x)

        x_addh[j] = x[j] + h
#         x_minh[j] = x[j]
        diff = (f(x_addh)-f(x_minh))/h
        a.append(diff)

    D_f = np.array(a).T

    return D_f

d_f(x01,h)

array([[  7.        ,  -1.        ,   4.        ],
       [ 12.        , 104.00000001,  44.        ],
       [  5.        ,  -1.        ,  -4.        ]])

In [169]:
x_new = x01
x_old = x01 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_ff(x_old, h)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [1. 2. 3.]
2  || [ 0.54640022  2.77839956 -0.26159934]
3  || [-2.45415806  7.13391919 -0.40311903]
4  || [-1.59789352  4.85549656 -0.37855891]
5  || [-1.42550899  3.00499549 -0.45122396]
6  || [-1.62369388  1.99376557 -0.67567635]
7  || [-1.6959843   1.97436748 -0.9033686 ]
8  || [-1.69054197  1.98302969 -0.88451334]
9  || [-1.69055076  1.98310724 -0.88455808]


array([-1.69055076,  1.98310724, -0.88455808])

In [170]:
x_new = x02
x_old = x02 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_ff(x_old, h)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [2. 2. 2.]
2  || [-0.45161433 -5.35484662 16.46776144]
3  || [ 0.21571445 -5.66419144  6.75907765]
4  || [ 0.2860225  -6.52534206  1.79625555]
5  || [ -0.25842433 -12.48641114  -1.61432047]
6  || [ -0.08943811 -11.02416489  -1.10705533]
7  || [  0.72874705 -46.46650732   4.67319982]
8  || [ -1.10344543 -88.17847799  -3.96519506]
9  || [ -4.72184359  90.40288921 -14.02459357]
10  || [ 60.66407356 781.25110444 153.76390782]
11  || [ 42.28276327 536.17000862 149.23652196]
12  || [ 30.59810458 370.23346689 143.89820944]
13  || [ 22.57810429 260.84268389 135.72467677]
14  || [ 16.41576382 189.59101136 123.47465875]
15  || [ 11.3918917  141.77169124 108.45300991]
16  || [  7.17310606 107.97622334  93.00719949]
17  || [ 3.26880686 82.68335124 79.18506533]
18  || [-3.57381441 60.40996032 71.59208263]
19  || [-0.29652693 51.0730305  52.39720333]
20  || [ 7.14370273 46.67010043 32.9760149 ]
21  || [ 4.20640586 35.09350389 28.76123765]
22  || [ 1.72601768 26.08645595 25.45907643]
23  || [-8

array([-1.,  3.,  1.])

In [171]:
# Newton meathod

def d_fn(x):
    x1, x2, x3 = x
    
    y1 = [3*x1**2+2*x2, 2*x1-x3, 2*x3-x2]
    y2 = [4*x1+2*x2**2, 4*x1*x2+3*(x2**2)*x3**2-2*x2*x3, 2*x2**3*x3-x2**2]
    y3 = [x2*x3+3*x1**2-x2**2, x1*x3-2*x1*x2, x1*x2-2*x3]

    D_f = np.array([y1,y2,y3])

    return D_f

d_fn(x01)

array([[  7.,  -1.,   4.],
       [ 12., 104.,  44.],
       [  5.,  -1.,  -4.]])

In [172]:
x_new = x01
x_old = x01 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_fn(x_old)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [1. 2. 3.]
2  || [ 0.5464  2.7784 -0.2616]
3  || [-2.45415693  7.13392934 -0.4031109 ]
4  || [-1.59789098  4.85550587 -0.37855094]
5  || [-1.42550577  3.00500187 -0.45121636]
6  || [-1.62369253  1.99376511 -0.67567152]
7  || [-1.69598454  1.97436714 -0.90336961]
8  || [-1.69054198  1.98302968 -0.88451335]
9  || [-1.69055076  1.98310724 -0.88455808]


array([-1.69055076,  1.98310724, -0.88455808])

In [173]:
x_new = x02
x_old = x02 - 1
i = 1

while np.linalg.norm(f(x_new))>tol_approx or np.linalg.norm(x_new-x_old)>tol_consec:
    print(f"{i}  ||", x_new)
    x_old = x_new
    
    # find linear equation
    D_f = d_fn(x_old)
    F_old = f(x_old)
    inverse_matrix = np.linalg.inv(D_f)
    
    x_new = x_old - np.dot(inverse_matrix, F_old)
    
    i+=1
    
x_new

1  || [2. 2. 2.]
2  || [-0.4516129  -5.35483871 16.46774194]
3  || [ 0.21571498 -5.66418297  6.75906942]
4  || [ 0.28602268 -6.52533239  1.79625249]
5  || [ -0.25842516 -12.48640211  -1.61432307]
6  || [ -0.08943887 -11.02414731  -1.10705913]
7  || [  0.72871695 -46.46470158   4.67295733]
8  || [ -1.10342368 -88.17585567  -3.96511265]
9  || [ -4.72168271  90.39754455 -14.0241217 ]
10  || [ 60.64551992 781.02052703 153.71581519]
11  || [ 42.27036841 536.00866542 149.19075827]
12  || [ 30.58967197 370.11954881 143.8551318 ]
13  || [ 22.57214085 260.76158673 135.6845358 ]
14  || [ 16.41142142 189.53209514 123.43811909]
15  || [ 11.38871926 141.72768054 108.42087148]
16  || [  7.1707915  107.9425363   92.97983131]
17  || [ 3.26696773 82.65698532 79.16250206]
18  || [-3.57937316 60.38566803 71.57890941]
19  || [-0.30297906 51.05268886 52.3872161 ]
20  || [ 7.10058343 46.62219944 33.01486144]
21  || [ 4.17426437 35.05855146 28.79399272]
22  || [ 1.69039443 26.05067296 25.50016358]
23  || [-9

array([-1.,  3.,  1.])

In [174]:
# def d_f(x,h):
#     a = []
#     for j in range(3):
#         x_addh = np.copy(x)
#         x_minh = np.copy(x)
#         x_addh[j] = x[j]+h
# #         print(x_addh[j])
#         x_minh[j] = x[j]-h
# #         print(x_minh[j])
# #         print(x_addh, x_minh, (f(x_addh)-f(x_minh))/2/h)
#         a.append((f(x_addh)-f(x_minh))/2/h)
    
#     res = np.array(a)
    
#     return res

    

    
# F_old = np.array(f(x01))
    
# inverse_matrix = np.linalg.inv(res_array)

# x_new = f(x01) - np.dot(inverse_matrix, F_old)

# Question 7

In [175]:
# using newtons method to bootstrap 0 rates
# at .5 to 2 year
r_5 = np.log(100/97.5)/.5

def f2(x):
    y = -100-5/32+2.4375*np.exp(-.5*r_5)+2.4375*np.exp(-(r_5+.5*x)/1.5)+2.4375*np.exp(-1.5*(.5*r_5+x)/1.5)+102.4375*np.exp(-2*x)
    return y

def d_f2(x):
    d_y = 2.4375*-.5/1.5*np.exp(-(r_5+.5*x)/1.5)+2.4375*(-1.5/1.5)*np.exp(-1.5*(.5*r_5+x)/1.5)+102.4375*-2*np.exp(-2*x)
    return d_y

x0 = .047
tol_approx = 1e-9
tol_consec = 1e-6

x_new = x0
x_old = x0-1

while abs(f2(x_new))>tol_approx or abs(x_new-x_old)>tol_consec:
    x_old = x_new
    x_new = x_old - f2(x_old)/d_f2(x_old)

    print(x_new, x_old)

r2 = x_new
r1_5 = (.5*r_5+r2)/1.5
r1 = (r_5+.5*r2)/1.5


0.047289178936440954 0.047
0.04728926180791532 0.047289178936440954


In [176]:
# using newtons method to bootstrap 0 rates
# at 2 to 3 year

def f3(x):
    y = -100-5/32+2.4375*np.exp(-.5*r_5)+2.4375*np.exp(-1*r1)+2.4375*np.exp(-1.5*r1_5)+2.4375*np.exp(-2*r2)+2.4375*np.exp(-2.5*.5*(x+r2))+102.4375*np.exp(-3*x)
    
    return y

def d_f3(x):
    d_y = 2.4375*-2.5*.5*np.exp(-2.5*.5*(r2+x))+102.4375*-3*np.exp(-3*x)
    return d_y

x0 = .047
tol_approx = 1e-9
tol_consec = 1e-6

x_new = x0
x_old = x0-1

while abs(f3(x_new))>tol_approx or abs(x_new-x_old)>tol_consec:
    x_old = x_new
    x_new = x_old - f3(x_old)/d_f3(x_old)

    print(x_new, x_old)
    
r3 = x_new
r2_5 = .5*r3 + .5*r2

0.047581370852375224 0.047
0.047581875451800544 0.047581370852375224


In [177]:
# at 3 to 5 year
def f5(x):
    r3_5 = (.5*x+1.5*r3)/2
    r4 = (x+r3)/2
    r4_5 = (1.5*x+.5*r3)/2
    r5 = x
    
    y = -99.6875+2.3125*np.exp(-.5*r_5)+2.3125*np.exp(-1*r1)+2.3125*np.exp(-1.5*r1_5)+2.3125*np.exp(-2*r2)+2.3125*np.exp(-2.5*r2_5)+2.3125*np.exp(-3*r3)+2.3125*np.exp(-3.5*r3_5)+2.3125*np.exp(-4*r4)+2.3125*np.exp(-4.5*r4_5)+102.3125*np.exp(-5*r5)
    
    return y

def d_f5(x):
    d_y = 2.3125*-.875*np.exp(-3.5*(.5*x+1.5*r3)/2)+2.3125*-2*np.exp(-2*(x+r3)/2)+2.3125*-3.375*np.exp(-4.5*(1.5*x+0.5*r3)/2)+102.3125*-5*np.exp(-5*x)
    return d_y


In [178]:
x0 = r3
tol_approx = 1e-9
tol_consec = 1e-6

x_new = x0
x_old = x0-1

while abs(f5(x_new))>tol_approx or abs(x_new-x_old)>tol_consec:
    x_old = x_new
    x_new = x_old - f5(x_old)/d_f5(x_old)

    print(x_new, x_old)
    
r5 = x_new
r4_5 = (1.5*x_new + .5*r3)/2
r4 = (x_new + r3)/2
r3_5 = (.5*x_new + 1.5*r3)/2

0.04630049937344657 0.047581875451800544
0.04630335677958187 0.04630049937344657
0.046303359378376534 0.04630335677958187


In [179]:
r5, r4_5, r4, r3_5

(0.046303359378376534,
 0.04662298839673253,
 0.04694261741508854,
 0.047262246433444545)

In [180]:
# at 5 to 10 year
def f10(x):
    r5_5 = .1*x + .9*r5
    r6   = .2*x + .8*r5
    r6_5 = .3*x + .7*r5
    r7   = .4*x + .6*r5
    r7_5 = .5*x + .5*r5
    r8   = .6*x + .4*r5
    r8_5 = .7*x + .3*r5
    r9   = .8*x + .2*r5
    r9_5 = .9*x + .1*r5
    r10  =    x
    
    
    y = -101.125+2.4375*np.exp(-.5*r_5)+2.4375*np.exp(-1*r1)+2.4375*np.exp(-1.5*r1_5)+2.4375*np.exp(-2*r2)+2.4375*np.exp(-2.5*r2_5)+2.4375*np.exp(-3*r3)+2.4375*np.exp(-3.5*r3_5)+2.4375*np.exp(-4*r4)+2.4375*np.exp(-4.5*r4_5)+2.4375*np.exp(-5*r5)+2.4375*np.exp(-5.5*r5_5)+2.4375*np.exp(-6*r6)+2.4375*np.exp(-6.5*r6_5)+2.4375*np.exp(-7*r7)+2.4375*np.exp(-7.5*r7_5)+2.4375*np.exp(-8*r8)+2.4375*np.exp(-8.5*r8_5)+2.4375*np.exp(-9*r9)+2.4375*np.exp(-9.5*r9_5)+102.4375*np.exp(-10*r10)
    
    return y

def d_f10(x):
    r5_5 = .1*x + .9*r5
    r6   = .2*x + .8*r5
    r6_5 = .3*x + .7*r5
    r7   = .4*x + .6*r5
    r7_5 = .5*x + .5*r5
    r8   = .6*x + .4*r5
    r8_5 = .7*x + .3*r5
    r9   = .8*x + .2*r5
    r9_5 = .9*x + .1*r5
    r10  =    x
    
    d_y = 2.4375*-5.5*.1*np.exp(-5.5*r5_5)+2.4375*-6*.2*np.exp(-6*r6)+2.4375*-6.5*.3*np.exp(-6.5*r6_5)+2.4375*-7*.4*np.exp(-7*r7)+2.4375*-7.5*.5*np.exp(-7.5*r7_5)+2.4375*-8*.6*np.exp(-8*r8)+2.4375*-8.5*.7*np.exp(-8.5*r8_5)+2.4375*-9*.8*np.exp(-9*r9)+2.4375*-9.5*.9*np.exp(-9.5*r9_5)+102.4375*-10*np.exp(-10*r10)
    
    return d_y

In [181]:
x0 = r5
tol_approx = 1e-9
tol_consec = 1e-6

x_new = x0
x_old = x0-1

while abs(f10(x_new))>tol_approx or abs(x_new-x_old)>tol_consec:
    x_old = x_new
    x_new = x_old - f10(x_old)/d_f10(x_old)

    print(x_new, x_old)
    
r5_5 = .1*x_new + .9*r5
r6   = .2*x_new + .8*r5
r6_5 = .3*x_new + .7*r5
r7   = .4*x_new + .6*r5
r7_5 = .5*x_new + .5*r5
r8   = .6*x_new + .4*r5
r8_5 = .7*x_new + .3*r5
r9   = .8*x_new + .2*r5
r9_5 = .9*x_new + .1*r5
r10  =    x_new

0.04677101007728406 0.046303359378376534
0.04677206531614356 0.04677101007728406
0.046772065321500064 0.04677206531614356


In [182]:
r_5, r2, r3, r5, r10

(0.05063561596857957,
 0.04728926180791532,
 0.047581875451800544,
 0.046303359378376534,
 0.046772065321500064)

# Question 8

In [491]:
# make dictioary for rates, bond prices and cashflows
r, B, cf = {}, {}, {}
for month in [2+m*6 for m in range(9)]:
    r[month], B[month], cf[month] = 'null', 'null', 'null'
for month in [m*6 for m in range (11)]:
    r[month], B[month], cf[month] = 'null', 'null', 'null'
for month in [4+m*6 for m in range(8)]:
    r[month], B[month], cf[month] = 'null', 'null', 'null'


r = dict(sorted(r.items()))
B = dict(sorted(B.items()))
cf = dict(sorted(cf.items()))

# instantiate known values
r[0] = .05
B[6], B[12], B[20], B[40], B[60] = 97.5, 100, 103, 102, 103
cf[6], cf[12], cf[20], cf[40], cf[60] = 0, 2.5, 3, 2.5, 2

In [492]:
# finding 6 month r0 
r[6] = np.log(100/B[6])/(6/12)

In [493]:
# in using newton method, need to get coef of x and constants
# create functions to get coefficient of x and constant

def interpolate(tlo, thi, t)-> float:
    """
    Generic function 
    used to interpolate linear rates for linear 
    tlo: tenure of lower rate
    thi: tenure of higher rate
    t  : tenure intermediate rate that is being 
    """

    rlo, tlo, thi = r[tlo], tlo, thi
    coef_r_hi = (t-tlo)/(thi-tlo)
    const  = (1-(t-tlo)/(thi-tlo))*rlo
    return coef_r_hi, const

def r_eval(tlo, thi, rhi, t)->float:
    rlo, tlo, rhi, thi = r[tlo], tlo , rhi, thi
    coef_r_hi, const = interpolate(tlo, thi, t)
    return coef_r_hi*rhi + const

def B_px(t)-> float:
    
    first = 1
    sum = 0
    
    for i in range(t,0,-6):
        if first:
            # print(t, i)
            sum += (100 + cf[t])*np.exp(-r[i]*i/12)
            first = 0
        else:
            sum += cf[t]*np.exp(-r[i]*i/12)
        # print(i)
    return sum
    

In [494]:
# assume linear rates btw 0 to 6 mths
# find 2 month zero rate
r[2] = r_eval(0, 6, r[6], 2)
r[4] = r_eval(0, 6, r[6], 4)

In [495]:
# get r12 by discounted cashflows from price
# B[12] = cf12*exp(-r6*.5) + (cf12+100)*exp(-r12*1, solve for r12
r[12] = np.log((100+2.5)/(100-2.5*np.exp(-r[6]*.5)))
# double check working 
B_px(12) == 100

True

In [496]:
# assume linear rates btw 6 to 12 mths
# find 8 month zero rate
r[8] = r_eval(6, 12, r[12], 8)
r[10] = r_eval(6, 12, r[12], 10)

In [497]:
# compute f(x) where f is value of a bond at time t
def f(x,t):
    first = True
    y = -B[t]
    tlo = max((key for key, value in r.items() if isinstance(value, float)), default=None)
    # print(tlo)
    
    for i in range(t,0,-6):
        if first:
            # print(t, i)
            if r[i] == "null":
                # compute guessed r interpolated
                r_temp = x
                y += (100 + cf[t])*np.exp(-r_temp*i/12)
            else: # else compute with existing r values in list
                y += (100 + cf[t])*np.exp(-r[i]*i/12)
            first = False
        else:
            if r[i] == "null":
                r_temp = r_eval(tlo, t, x, i)
                y += cf[t]*np.exp(-r_temp*i/12)
            else:
                y += cf[t]*np.exp(-r[i]*i/12)
        # print(i, y, first)
        
    return y


def d_f(x,t):
    first = True
    y = 0
    tlo = max((key for key, value in r.items() if isinstance(value, float)), default=None)
    
    for i in range(t,0,-6):
        if r[i] == "null":
            if first:
                # print(t, i)
                # compute guessed r interpolated
                r_temp = x
                y += (100 + cf[t])*(-i/12)*np.exp(-r_temp*i/12)
                first = False
            else:
                r_coef, _ = interpolate(tlo, t, i)
                r_temp = r_eval(tlo, t, x, i)
                y += cf[t]*(-r_coef*i/12)*np.exp(-r_temp*i/12)
        # skip values when they are present
        # print(i, y, first)
        
    return y

# B[20] = cf20*exp(-r2*2/12) 
    # +(cf20)*exp(-r8*8/12)
    # +(cf20)*exp(-r14*14/12)
    # +(cf20+100)*exp(-r20*20/12)
# use a for loop to compute known alues for f(x), t =2, 8 and also compute for other interpolated rates

def zero_curve(x_approx, t_approx, tol_approx=1e-9, tol_consec = 1e-6)->float:
    x0 = x_approx
    
    t_new = t_approx
    x_new = x0
    x_old = x0-1
    
    while abs(f(x_new, t_new))>tol_approx or abs(x_new-x_old)>tol_consec:
        x_old = x_new
        x_new = x_old - f(x_new, t_new)/d_f(x_new, t_new)
    
        print(x_new, x_old)

    return x_new

In [498]:
r[20] = zero_curve(r[12],20)
r[14] = r_eval(12,20,r[20],14)
r[16] = r_eval(12,20,r[20],16)
r[18] = r_eval(12,20,r[20],18)

0.05297184363493467 0.04936960030281203
0.05298265366301247 0.05297184363493467
0.052982653759974574 0.05298265366301247


In [499]:
# Check my answer
B_px(20) == B[20]

True

In [500]:
[x for x in range(40-2,20,-2)]

[38, 36, 34, 32, 30, 28, 26, 24, 22]

In [501]:
r[40] = zero_curve(r[20],40)
for m in range(40-2,20,-2):
    r[m] = r_eval(20,40,r[40],m)
B_px(40) - B[40] < 1e-10

0.045228347234849996 0.052982653759974574
0.045325667009868766 0.045228347234849996
0.04532568260711439 0.045325667009868766


True

In [502]:
r[60] = zero_curve(r[40],60)
for m in range(60-2,40,-2):
    r[m] = r_eval(40,60,r[60],m)
B_px(60)-B[60] < 1e-9

0.03167780004393844 0.04532568260711439
0.032118488293693424 0.03167780004393844
0.03211896969755868 0.032118488293693424


True

In [503]:
B_px(60)

103.00000000025413

In [504]:
r[0], r[2], r[8], r[20], r[40], r[60]

(0.05,
 0.05021187198952653,
 0.05021361074665706,
 0.052982653759974574,
 0.04532568260711439,
 0.03211896969755868)

# Appendix:
## Helper functions

In [247]:
max_key = None
max_value = float('-inf')

# for key, value in r.items():
#     if isinstance(value, float) and (max_key is None or key > max_key):
#         max_key = key
#         max_value = value

max_key = max((key for key, value in r.items() if isinstance(value, float)), default=None)

max_key

12